# ABOUT:
- We fine tune a pretrained model with Transformers architecture 

In [1]:
model_checkpoint =  "albert-xlarge-v1"
model_checkpoint = "bert-base-uncased"
num_labels = 2
batch_size = 20

## load_from_disk

In [2]:
from datasets import load_from_disk

In [3]:
tokenized_train_Dataset = load_from_disk(r"C:\Users\tanch\Documents\Coding Competitions\Kaggle\Natural Language Processing with Disaster Tweets (local)\data\tokenized train set")
tokenized_train_Dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'id', 'input_ids', 'new_input_string', 'target', 'token_type_ids'],
        num_rows: 6851
    })
    test: Dataset({
        features: ['attention_mask', 'id', 'input_ids', 'new_input_string', 'target', 'token_type_ids'],
        num_rows: 762
    })
})

In [4]:
from datasets import ClassLabel, Sequence
import pandas as pd
from IPython.display import display, HTML
def show_elements(dataset, indices = range(5)):
    df = pd.DataFrame(dataset[indices])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [5]:
tokenized_train_Dataset.rename_column_("target","label")

In [6]:
tokenized_train_Dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'id', 'input_ids', 'new_input_string', 'token_type_ids', 'label'],
        num_rows: 6851
    })
    test: Dataset({
        features: ['attention_mask', 'id', 'input_ids', 'new_input_string', 'token_type_ids', 'label'],
        num_rows: 762
    })
})

In [7]:
show_elements(tokenized_train_Dataset["train"])

,attention_mask,id,input_ids,new_input_string,label,token_type_ids
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",4874,"[101, 15044, 1012, 2156, 2122, 4364, 4285, 1996, 2392, 3329, 2041, 10578, 1996, 4065, 1998, 9419, 1029, 4445, 2079, 1045, 999, 1001, 7294, 1001, 7170, 1004, 23713, 1025, 15044, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",explode. See these guys reaching the front foot out loading the shoulders and spinning? Neither do I! #hitting #load&amp;explode,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",9915,"[101, 4390, 1012, 1037, 2158, 1037, 2450, 7472, 1004, 23713, 1025, 2148, 21358, 23778, 2319, 4390, 1011, 2006, 1001, 5096, 3653, 8551, 2121, 1001, 1996, 8237, 23805, 2078, 1011, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",trouble. A man a woman Romance &amp; South Afrikaan Trouble - on # sale PreOrder #theBargain -,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",962,"[101, 15347, 1012, 1030, 6520, 5558, 7295, 28522, 4371, 11245, 2041, 15347, 1996, 20930, 6520, 1999, 1996, 2061, 2705, 19650, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",blaze. @DJJOHNBLazE shout out blaze the hottest DJ in the Sothwest,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",10737,"[101, 21056, 1012, 21056, 1005, 9530, 23633, 2135, 4484, 1005, 2004, 2013, 1049, 2232, 24434, 2692, 1024, 6027, 7610, 1024, 14766, 1998, 1996, 2945, 1997, 2216, 2040, 2020, 1012, 1012, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## AutoTokenizer

In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

## AutoModelForSequenceClassification

In [9]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoConfig,DataCollatorWithPadding

In [10]:
model  = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, 
                                                            num_labels=num_labels)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [11]:
print(model.device)
model.to("cuda")

cpu


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

## TrainingArguments and Trainer

In [12]:
args = TrainingArguments(
    "test-glue",
    evaluation_strategy = "epoch",                           
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,                               # best model may not be the model at the end of training, thus this param enables us to save any best model during training
    # metric_for_best_model=metric_name,
)

In [13]:
trainer = Trainer(
    model,
    args,
    train_dataset= tokenized_train_Dataset['train'],           # we could replaced with a Dataset shard to speed up search
    eval_dataset=tokenized_train_Dataset['test'],
    # data_collator = collator
)

In [14]:
# import mlflow
# mlflow.end_run()

In [15]:
# we see that the model overfits after the first epoch
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: new_input_string, id.
***** Running training *****
  Num examples = 6851
  Num Epochs = 5
  Instantaneous batch size per device = 20
  Total train batch size (w. parallel, distributed & accumulation) = 20
  Gradient Accumulation steps = 1
  Total optimization steps = 1715


Epoch,Training Loss,Validation Loss
1,No log,0.397001
2,0.423800,0.404179
3,0.321800,0.451131
4,0.321800,0.501079
5,0.245600,0.495903


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: new_input_string, id.
***** Running Evaluation *****
  Num examples = 762
  Batch size = 20
Saving model checkpoint to test-glue\checkpoint-343
Configuration saved in test-glue\checkpoint-343\config.json
Model weights saved in test-glue\checkpoint-343\pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: new_input_string, id.
***** Running Evaluation *****
  Num examples = 762
  Batch size = 20
Saving model checkpoint to test-glue\checkpoint-686
Configuration saved in test-glue\checkpoint-686\config.json
Model weights saved in test-glue\checkpoint-686\pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: new_input_string, i

TrainOutput(global_step=1715, training_loss=0.3179643836730423, metrics={'train_runtime': 994.5053, 'train_samples_per_second': 34.444, 'train_steps_per_second': 1.724, 'total_flos': 2250220089234000.0, 'train_loss': 0.3179643836730423, 'epoch': 5.0})

In [16]:
# check if the best model was indeed loaded - 0.39 eval loss corresponds to first epoch. Yes.
trainer.evaluate(tokenized_train_Dataset['test'])

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: new_input_string, id.
***** Running Evaluation *****
  Num examples = 762
  Batch size = 20


{'eval_loss': 0.3970007002353668,
 'eval_runtime': 6.8004,
 'eval_samples_per_second': 112.052,
 'eval_steps_per_second': 5.735,
 'epoch': 5.0}

In [ ]:
model.save_pretrained(r"C:\Users\tanch\Documents\Coding Competitions\Kaggle\Natural Language Processing with Disaster Tweets (local)\models\bert uncased tuned")

## Check sanity of predictions

In [19]:
output = trainer.predict(tokenized_train_Dataset['test'])

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: new_input_string, id.
***** Running Prediction *****
  Num examples = 762
  Batch size = 20


In [37]:
def compute_accuracy(logits, labels):
    y_pred = np.argmax(logits, axis=1)
    return {"Accuracy":sum(labels==y_pred)/len(y_pred)}

In [38]:
compute_accuracy(output.predictions,tokenized_train_Dataset['test']['label'])

<IPython.core.display.Javascript object>

{'Accuracy': 0.8398950131233596}

## side note: compute_metrics
- if we use compute_metrics parameter to choose best model we need
    1. to create a function 
    1. that takes the model output 
    2. and returns a dictionary {'matthews_correlation': 0.6752283559689948}


In [ ]:
# compute_metrics function 
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

In [31]:
# model output 
output

PredictionOutput(predictions=array([[ 1.4946233 , -0.7929134 ],
       [ 0.96645635, -0.28999025],
       [ 1.2729925 , -0.5768309 ],
       ...,
       [ 1.464928  , -0.59932643],
       [-2.0104194 ,  1.2533122 ],
       [ 1.1098032 , -0.17768256]], dtype=float32), label_ids=array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0

In [30]:
# compute_metrics output
metric.compute(predictions=np.argmax(output.predictions, axis=1), references=output.label_ids)

<IPython.core.display.Javascript object>

{'matthews_correlation': 0.6752283559689948}